In [61]:
from keras.models import Sequential , Model
from keras.layers import Dense , Conv2D , MaxPool2D , Flatten
from keras.layers import Input

from keras.utils import to_categorical

from keras.optimizers import Adam

from glob import glob

In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy


%matplotlib inline

In [53]:
BATCH_SIZE = 32

PATH = '../dataset/CIFAR10/train/'

IMAGES_NAME = glob(PATH+'*')

WIDTH = 32
HEIGHT = 32
CHANNEL = 3

IMAGE_SHAPE = (WIDTH , HEIGHT , CHANNEL)

CLASS_NUM = 10 #CIFAR10

In [25]:

LABEL2INDEX = {'frog':0,'truck':1,'deer':2,'automobile':3,'bird':4,'horse':5,'ship':6,'cat':7,'dog':8,'airplane':9}

INDEX2LABEL = {value:key for key,value in LABEL2INDEX.items()}

labels = pd.read_csv('../dataset/CIFAR10/trainLabels.csv')


In [11]:
def load_image():
    images = []
    
    for i in IMAGES_NAME:
        image = scipy.misc.imread(i , mode='RGB').astype(np.float)
        
        images.append(image)
    
    images = np.array(images)/127.5 - 1
    
    return images

In [27]:
def load_label():
    labels = pd.read_csv('../dataset/CIFAR10/trainLabels.csv')

    return to_categorical(labels.label.map(LABEL2INDEX).values , num_classes=10)

In [12]:
images = load_image()

In [28]:
labels = load_label()

In [29]:
#=============

In [32]:
from keras.applications import VGG16

In [33]:
vgg = VGG16()

In [34]:
vgg.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [35]:
vgg.input_shape

(None, 224, 224, 3)

In [36]:
vgg.output_shape

(None, 1000)

In [79]:
input_image = Input(shape=IMAGE_SHAPE)

for i in range(1 , 20):
    #print(i)
    hidden_layer = vgg.layers[i]
    hidden_layer.trainable = False
    input_image = hidden_layer(input_image)
    
#经过VGG16的Flatten
h = Dense(4096 , activation='relu')(input_image)
h = Dense(4096 , activation='relu')(h)
pred = Dense(CLASS_NUM , activation='softmax')(h)

In [82]:
model_vgg = Model(inputs = input_image , outputs = pred)

AttributeError: 'Model' object has no attribute 'name'

In [83]:
input_image = Input(shape=IMAGE_SHAPE)


hidden_layer = vgg.layers[1]
hidden_layer.trainable = False
input_image = hidden_layer(input_image)
    
#经过VGG16的Flatten
h = Flatten()(input_image)
h = Dense(4096 , activation='relu')(h)
h = Dense(4096 , activation='relu')(h)
pred = Dense(CLASS_NUM , activation='softmax')(h)

In [85]:
model_vgg = Model(inputs = input_image , outputs = pred , name='d')

AttributeError: 'Model' object has no attribute 'name'

In [72]:
#============

In [54]:
model = Sequential()
model.add(Conv2D(filters=64 , input_shape=IMAGE_SHAPE , kernel_size=3 , strides=1 , padding='same' , activation='relu'))
model.add(Conv2D(filters=64 , kernel_size=3 , strides=1 , padding='same' , activation='relu'))
model.add(MaxPool2D(pool_size=(2,2) , strides=(2,2)))

model.add(Conv2D(filters=128 , kernel_size=3 , strides=1 , padding='same' , activation='relu'))
model.add(Conv2D(filters=128 , kernel_size=3 , strides=1 , padding='same' , activation='relu'))
model.add(MaxPool2D(pool_size=(2,2) , strides=(2,2)))

model.add(Conv2D(filters=256 , kernel_size=3 , strides=1 , padding='same' , activation='relu'))
model.add(Conv2D(filters=256 , kernel_size=3 , strides=1 , padding='same' , activation='relu'))
model.add(Conv2D(filters=256 , kernel_size=3 , strides=1 , padding='same' , activation='relu'))
model.add(MaxPool2D(pool_size=(2,2) , strides=(2,2)))

model.add(Conv2D(filters=512 , kernel_size=3 , strides=1 , padding='same' , activation='relu'))
model.add(Conv2D(filters=512 , kernel_size=3 , strides=1 , padding='same' , activation='relu'))
model.add(Conv2D(filters=512 , kernel_size=3 , strides=1 , padding='same' , activation='relu'))
model.add(MaxPool2D(pool_size=(2,2) , strides=(2,2)))

model.add(Conv2D(filters=512 , kernel_size=3 , strides=1 , padding='same' , activation='relu'))
model.add(Conv2D(filters=512 , kernel_size=3 , strides=1 , padding='same' , activation='relu'))
model.add(Conv2D(filters=512 , kernel_size=3 , strides=1 , padding='same' , activation='relu'))
model.add(MaxPool2D(pool_size=(2,2) , strides=(2,2)))

model.add(Flatten())

model.add(Dense(4096 , activation='relu'))
model.add(Dense(4096 , activation='relu'))
model.add(Dense(CLASS_NUM , activation='softmax'))

In [89]:
model.layers[0].set_weights([w for w in vgg.layers[1].weights])

ValueError: setting an array element with a sequence.

In [90]:
model.layers[0].weights

[<tf.Variable 'conv2d_55/kernel:0' shape=(3, 3, 3, 64) dtype=float32_ref>,
 <tf.Variable 'conv2d_55/bias:0' shape=(64,) dtype=float32_ref>]

In [91]:
vgg.layers[1].weights

[<tf.Variable 'block1_conv1/kernel:0' shape=(3, 3, 3, 64) dtype=float32_ref>,
 <tf.Variable 'block1_conv1/bias:0' shape=(64,) dtype=float32_ref>]

In [57]:
model.compile(optimizer=Adam() , loss='categorical_crossentropy' , metrics=['accuracy'])

In [58]:
model.fit(x=images , y=labels , batch_size=32 , epochs=10 , validation_split=0.1)

Train on 45000 samples, validate on 5000 samples
Epoch 1/10
45000/45000 [==============================] - 111s 2ms/step - loss: 2.3029 - acc: 0.0970 - val_loss: 2.3028 - val_acc: 0.0976
Epoch 2/10
32192/45000 [====================>.........] - ETA: 30s - loss: 2.3028 - acc: 0.0996

KeyboardInterrupt: 